## Динамика логитов

In [1]:
!pip install datasets jaxtyping better_abc pillow wandb circuitsvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 53.7 MB/s eta 0:00:00


#### Связи проекта

In [2]:
import sys
from pathlib import Path

In [3]:
path = "."
# for kaggle
# path = "/kaggle/input/logitlens32/LogitLens/research/logits-dynamics/notebook"

ROOT = Path(path).absolute().parent.parent.parent

DATA = ROOT / "data"

In [4]:
if ROOT not in sys.path:
    sys.path.append(str(ROOT))

#### Импорт библиотек

In [5]:
import requests
import torch
from PIL import Image
import circuitsvis as cv
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
from math import ceil
import json

import transformer_lens.utils as utils
from transformer_lens import HookedTransformer

from research.utils import get_data_images, plot_data, plot_data_attention, plot_images

#### Логиты

Создаем модель

In [6]:
torch.set_grad_enabled(False)

In [7]:
device = utils.get_device()

In [8]:
model_name = "llava-hf/llava-1.5-7b-hf"
last_layer = 32

In [9]:
model = HookedTransformer.from_pretrained(model_name, device=device)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

ValueError: Unrecognized image processor in llava-hf/llava-1.5-7b-hf. Should have a `image_processor_type` key in its preprocessor_config.json of config.json, or one of the following `model_type` keys in its config.json: align, beit, bit, blip, blip-2, bridgetower, chameleon, chinese_clip, clip, clipseg, conditional_detr, convnext, convnextv2, cvt, data2vec-vision, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, donut-swin, dpt, efficientformer, efficientnet, flava, focalnet, fuyu, git, glpn, grounding-dino, groupvit, hiera, idefics, idefics2, idefics3, ijepa, imagegpt, instructblip, instructblipvideo, kosmos-2, layoutlmv2, layoutlmv3, levit, llava, llava_next, llava_next_video, llava_onevision, mask2former, maskformer, mgp-str, mllama, mobilenet_v1, mobilenet_v2, mobilevit, mobilevitv2, nat, nougat, oneformer, owlv2, owlvit, paligemma, perceiver, pix2struct, pixtral, poolformer, pvt, pvt_v2, qwen2_vl, regnet, resnet, rt_detr, sam, segformer, seggpt, siglip, swiftform

Загружаем изображения из датасета COCO

In [ ]:
COCO = DATA / "coco_ann2017"
IMAGE_URL_FILE = COCO / "images_urls.json"

In [ ]:
with open(IMAGE_URL_FILE, mode="r") as file:
    dict_images = json.load(file)

images = [Image.open(requests.get(url, stream=True).raw) for url in dict_images["urls"]]
class_images = dict_images["labels"]
prompt = "USER: <image>\nWhat is shown in the picture? ASSISTANT: The picture shows a"

Отобразим изображения

In [ ]:
plot_images(images)

Соберем данные для каждого из изображений: токены, вероятность, энтропия, косинусное расстояние между последним слоем и промежуточными слоями модели и внимание.

In [ ]:
data, data_attention = get_data_images(model, class_images, prompt, images, last_layer)

График зависимости вероятности предсказания правильного токена от номера слоя.

In [ ]:
plot_data(data, class_images, data_id=1, xlabel='n_layer', ylabel='prob')

График зависимости энтропии от номера слоя.

Энтропия можно считать характеристикой уверенности модели в ответе.

In [ ]:
plot_data(data, class_images, data_id=2, dy=0.3, xlabel='n_layer', ylabel='entropy')

График зависимости косинусное расстояние между последним слоем и промежуточными слоями модели от номера слоя.

In [ ]:
plot_data(data, class_images, data_id=3, xlabel='n_layer', ylabel='cos_dist')

Графики зависимости усредненного внимания от номера слоя.

Внимание в изображении

In [ ]:
plot_data_attention(data_attention, class_images, label="image", xlabel='n_layer', ylabel='attention_mean')

Внимание в тексте

In [ ]:
plot_data_attention(data_attention, class_images, label="text", xlabel='n_layer', ylabel='attention_mean')

Внимание между текстом и изображением

In [ ]:
plot_data_attention(data_attention, class_images, label="cross", xlabel='n_layer', ylabel='attention_mean')